In [15]:
from azureml.core import Experiment, Environment, Workspace, Datastore, Dataset, Model, ScriptRunConfig, Run
import os
import glob
# get the current workspace
ws = Workspace.from_config()

In [18]:
%cd Satellite_ComputerVision
!git pull
%cd ..

Already up-to-date.
/mnt/batch/tasks/shared/LS_root/mounts/clusters/test-compute-instance/code/Users/mevans


In [16]:
# access our registered data share containing image data in this workspace
datastore = Datastore.get(workspace = ws, datastore_name = 'solardatablob')

cpk_train_path = (datastore, 'CPK_solar/data/training/')
cpk_eval_path = (datastore, 'CPK_solar/data/eval/')

nc_train_path = (datastore, 'NC_solar/data/training/')
nc_eval_path = (datastore, 'NC_solar/data/eval/')

test_path = (datastore, 'CPK_solar/data/predict/testpred5')

# train_dataset = Dataset.File.from_files(path = [cpk_train_path])
# eval_dataset = Dataset.File.from_files(path = [cpk_eval_path])

# nc_train_dataset = Dataset.File.from_files(path = [nc_train_path])
# nc_eval_dataset = Dataset.File.from_files(path = [nc_eval_path])

# when we combine datasets the selected directories and relative paths to the datastore are brought in
# mount folder
# |-cddatafilestore
# | |-GEE
# | | |-training
# | | |-eval
# | |-Onera
# | | |-training
# | | |-eval

train_dataset = Dataset.File.from_files(path = [cpk_train_path, nc_train_path])
eval_dataset = Dataset.File.from_files(path = [cpk_eval_path, nc_eval_path])
test_dataset = Dataset.File.from_files(path = [test_path])

In [18]:
# FInd the run corresponding to the model we want to register
# run_id = 'solar-nc-cpk_1624989679_f59da7cf'
run_id = 'solar-nc-cpk_1684259900_b71cc594'
run = ws.get_run(run_id)

In [17]:
model_name = 'solar_May23'

In [19]:
model = run.register_model(model_name=model_name,
                           tags=run.tags,
                           description = 'UNET model delineating ground mounted solar arrays in S2 imagery. Trained on multi-season data from Chesapeake Bay and NC',
                           model_path='outputs/',
                           model_framework = 'Tensorflow',
                           model_framework_version= '2.0',
                           datasets = [('training', train_dataset), ('evaluation', eval_dataset), ('testing', test_dataset)])
print(model.name, model.id, model.version, sep='\t')

solar_May23	solar_May23:2	2


In [17]:
# use the azure folder as our script folder
source = 'Satellite_ComputerVision'
util_folder = 'utils'
script_folder = f'{source}/azure'
script_file = 'train_solar.py'

In [20]:
# define the compute target
ws.compute_targets
mevansGPU = ws.compute_targets['mevansGPU']

In [22]:
experiment_name = 'solar-nc-cpk'
exp = Experiment(workspace = ws, name = experiment_name)

In [21]:
custom_env = Environment.from_docker_image(
    name = 'tf_training',
    image = 'mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.2-cudnn8-ubuntu20.04:20221010.v1',
    container_registry=None,
    conda_specification='/mnt/batch/tasks/shared/LS_root/mounts/clusters/mevans1/code/Users/mevans/Solar_UNet/conda_env-copy.yml',
    pip_requirements=None)

In [23]:
RESPONSE = 'landcover'
args = [
    '--train_data', train_dataset.as_mount(),
    '--eval_data', eval_dataset.as_mount(),
    '--test_data', test_dataset.as_mount(),
    '--model_id', model_name,
    '--weights', '[1.0, 1.0]',
    '--bias', 0,
    '-lr', 0.0005,
    '--epochs', 200,
    '--epoch_start', 150,
    '--batch', 16,
    '--size', 11020,
    '--kernel_size', 256,
    '--response', f'{RESPONSE}',
    '--bands', '["B2", "B3", "B4", "B8", "B11", "B12"]',
    '--splits', '[0]']

src = ScriptRunConfig(source_directory='azure',
                      script='train_solar.py', 
                      arguments=args,
                      compute_target=mevansGPU,
                      environment=custom_env)

In [24]:
#  run the training job
run = exp.submit(config=src, tags = dict({'splits':'None', 'model':'Unet', 'dataset':'NC CPK S2', 'normalization':'S2 moments', 'epochs':'150-200'}))
run

Experiment,Id,Type,Status,Details Page,Docs Page
solar-nc-cpk,solar-nc-cpk_1684345980_10f4f4d2,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


Bad pipe message: %s [b'8\xde(\xabdJN#r\xf9\x05\xd7\xe1h\xee\x83\xac\xda \xe7\x80\x7fCu\x10\xe4\xd9\x94\xc0\xbdj\xc4\xd9\xcb\x18/\xde\x06M\xfe\xff\xe6\xd0\x9a\xf4m\x08\xcb\x8f\x93p\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04']
Bad pipe message: %s [b"\x86o~1o\x1bu.\xf9\xdd\xc4\xde0\x92\xfd\x18$\x8d\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x0